Домашнее задание 

• Выберите любую новость с любого новостного портала
• Проведите парсинг текста новости
• Очистите текст от стоп-слов
• Оцените тональность новости с помощью библиотеки dostoevsky
• В случае проблем с установкой dostoevsky проведите лемматизацию текста и выведите 5
самых частых слов в новости
• Срок – до 25 ноября

In [1]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = "https://ria.ru/20241122/laos-1985287875.html" # Это плохая новость. 

page = requests.get(url)
print(page.status_code)

200


In [3]:
soup = BeautifulSoup(page.text, "html.parser")

In [4]:
soup.title.string

'В Лаосе шесть туристов отравились алкоголем - РИА Новости, 23.11.2024'

In [5]:
# Текст новости разбит на части и содержится в объектах class=... 
# Названия классов на разных сайтах разные, надо смотреть исходник страницы. 

soup.find(class_ = 'article__block')

<div class="article__block" data-type="text"><div class="article__text"><strong>МОСКВА, 22 ноя — РИА Новости.</strong> Шесть иностранных туристов погибли из-за отравления алкоголем в городе Ванг-Вьенг в Лаосе, пишет местная газета <a href="https://www.vientianetimes.org.la/" rel="nofollow noopener" target="_blank">Vientiane Times</a>. </div></div>

In [6]:
# Циклом перебираем блоки и собираем в переменную text 
# Eсли class_="article__article" - пропускаем блок 

text = ''
for block in soup.find_all(class_ = 'article__block'):
  if block.find('div', class_="article__article"):
    pass
  else:
    text = ''.join([text, block.get_text()])

# Чистый текст новости 
text

'МОСКВА, 22 ноя — РИА Новости. Шесть иностранных туристов погибли из-за отравления алкоголем в городе Ванг-Вьенг в Лаосе, пишет местная газета Vientiane Times. «"Сотрудники правоохранительных органов проводят детальное расследование по факту, предположительно, плохих напитков, от которых, по предварительным данным, погибли шесть иностранных граждан, отдыхавших в Ванг-Вьенге", — говорится в публикации.Жертвами массового отравления стали граждане Австралии, Дании, Великобритании и США. Кроме того, в больницы попали еще девять человек. Предположительно, они отравились некачественным алкоголем, содержащим метанол. Сообщается, что власти Лаоса проводят детальное расследование для выявления причины трагедии. В субботу делегация сотрудников Министерства информации, культуры и туризма провинции Вьентьян совместно со СМИ прибудут в Ванг-Вьенг, чтобы разобраться в ситуации и обнародовать результаты расследования.'

In [ ]:
# Надо разобрать строку с текстом на слова 

In [9]:
import nltk
# Библиотека nltk на Дебиан установилась, работает! 

In [153]:
# А библиотека dostoevsky на Дебиан не ставится, Дебиан 12 не знает про такую библиотеку. 
# И pip install выдаёт ошибку: 
%pip install dostoevsky 

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.11/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai


Промудохался несколько часов, пересмотрел кучу статей в Интернете, но я это сделал !!! 

В результате всё оказалось просто: 

1. Скачал файл dostoevsky-0.6.0.tar.gz с этой страницы: https://pypi.org/project/dostoevsky/#files
2. Распаковал архив. 
3. В терминале Линукса ввёл команды: 
$ python3 -m venv ~/Загрузки/dostoevsky-0.6.0
$ ~/Загрузки/dostoevsky-0.6.0/bin/pip install dostoevsky
$ sudo python3 -m dostoevsky download fasttext-social-network-model

Ещё пришлось доустановить пакет python3-fasttext, т.к. в Jupyter выдавалась ошибка. 
$ sudo apt install python3-fasttext 


In [154]:
# УРА, РАБОТАТЕ !!! 

# Пример со страницы https://pypi.org/project/dostoevsky/ 

from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()
tokens = tokenizer.split('всё очень плохо')  # [('всё', None), ('очень', None), ('плохо', None)]

model = FastTextSocialNetworkModel(tokenizer=tokenizer)

messages = [
    'привет',
    'я люблю тебя!!',
    'малолетние дебилы'
]

results = model.predict(messages, k=2)

for message, sentiment in zip(messages, results):
    # привет -> {'speech': 1.0000100135803223, 'skip': 0.0020607432816177607}
    # люблю тебя!! -> {'positive': 0.9886782765388489, 'skip': 0.005394937004894018}
    # малолетние дебилы -> {'negative': 0.9525841474533081, 'neutral': 0.13661839067935944}]
    print(message, '->', sentiment)

# Хоть и с варнингом, но работатет! 

привет -> {'speech': 1.0000100135803223, 'skip': 0.0020607432816177607}
я люблю тебя!! -> {'positive': 0.9886782765388489, 'skip': 0.005394937004894018}
малолетние дебилы -> {'negative': 0.9525841474533081, 'neutral': 0.13661839067935944}


In [ ]:
# Продожаем дальше 

In [10]:
# Удаление стоп-слов 

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_ru = stopwords.words("russian")
stopwords_ru[0:12]

[nltk_data] Downloading package stopwords to /home/slava/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а']

In [11]:
import re

In [12]:
words = [word.lower() for word in re.split(';|\.| ', text) if word.lower() not in stopwords_ru]
text = ' '.join(words)

text

'москва, 22 ноя — риа новости  шесть иностранных туристов погибли из-за отравления алкоголем городе ванг-вьенг лаосе, пишет местная газета vientiane times  «"сотрудники правоохранительных органов проводят детальное расследование факту, предположительно, плохих напитков, которых, предварительным данным, погибли шесть иностранных граждан, отдыхавших ванг-вьенге", — говорится публикации жертвами массового отравления стали граждане австралии, дании, великобритании сша  кроме того, больницы попали девять человек  предположительно, отравились некачественным алкоголем, содержащим метанол  сообщается, власти лаоса проводят детальное расследование выявления причины трагедии  субботу делегация сотрудников министерства информации, культуры туризма провинции вьентьян совместно сми прибудут ванг-вьенг, разобраться ситуации обнародовать результаты расследования '

In [14]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

data = model.predict([text])
data

[{'neutral': 0.6442351341247559,
  'negative': 0.22816647589206696,
  'skip': 0.0980893224477768,
  'positive': 0.04337579756975174,
  'speech': 0.03309597820043564}]

In [15]:
df = pd.DataFrame(data)

df.insert(0, 'title', soup.title.string)
df

,title,neutral,negative,skip,positive,speech
0,В Лаосе шесть туристов отравились алкоголем - ...,0.644235,0.228166,0.098089,0.043376,0.033096


In [ ]:
# Вот такая ерунда получается. 
# Библиотека выдаёт, что новость в целом нейтральная (0.644), негатива совсем немного (0.228). 

In [ ]:
# Будем тестировать другую новость 

In [17]:
# Другая новость с другого сайта 
url2 = "https://www.kommersant.ru/doc/7312132" # Нормальный текст, позитивного содержания 

page2 = requests.get(url2)
print(page2.status_code)

200


In [18]:
soup2 = BeautifulSoup(page2.text, "html.parser")
soup2.title.string

'Портреты русской цивилизации: Великий Новгород'

In [19]:
soup2.find(class_ = 'doc__text')

<p class="doc__text doc__intro">Этот город выстроен так, будто находящийся в его центре кремль — невероятное сокровище, которое нельзя осквернять присутствием простых человеческих радостей, подпуская их ближе чем на километр. Только высокая духовность новгородской Софии, только думы о русской истории вокруг памятника тысячелетия. Тот же регламент — пространственная почтительность к древнерусским памятникам — соблюдается по всему городу. Это уникальная городская структура, и в известном смысле именно она кажется самой интересной чертой города.</p>

In [20]:
text2 = ''
for block in soup2.find_all(class_ = 'doc__text'):
    text2 = ''.join([text2, block.get_text()])

# Чистый текст новости 
text2

'Этот город выстроен так, будто находящийся в его центре кремль — невероятное сокровище, которое нельзя осквернять присутствием простых человеческих радостей, подпуская их ближе чем на километр. Только высокая духовность новгородской Софии, только думы о русской истории вокруг памятника тысячелетия. Тот же регламент — пространственная почтительность к древнерусским памятникам — соблюдается по всему городу. Это уникальная городская структура, и в известном смысле именно она кажется самой интересной чертой города.Текст: Григорий РевзинВ 1862\xa0году в Новгородском кремле был открыт памятник «Тысячелетие России» работы Михаила Осиповича Микешина. Поскольку в нем в краткой скульптурной форме резюмирована история государства, от призвания варягов до Крымской войны, то его используют как объект обязательного экскурсионного показа и рассказа об этой истории, а его изображения часто встречаются на разнообразной печатной продукции — от школьных дневников до маек. Это очень известное произведени

In [21]:
# Удаление стоп-слов 

words2 = [word.lower() for word in re.split(';|\.| ', text2) if word.lower() not in stopwords_ru]
text2 = ' '.join(words2)

text2

'город выстроен так, находящийся центре кремль — невероятное сокровище, которое осквернять присутствием простых человеческих радостей, подпуская ближе километр  высокая духовность новгородской софии, думы русской истории вокруг памятника тысячелетия  регламент — пространственная почтительность древнерусским памятникам — соблюдается всему городу  это уникальная городская структура, известном смысле именно кажется самой интересной чертой города текст: григорий ревзинв 1862\xa0году новгородском кремле открыт памятник «тысячелетие россии» работы михаила осиповича микешина  поскольку нем краткой скульптурной форме резюмирована история государства, призвания варягов крымской войны, используют объект обязательного экскурсионного показа рассказа истории, изображения часто встречаются разнообразной печатной продукции — школьных дневников маек  это очень известное произведение  менее известен контекст, котором оно расположено  оно стоит старой дороге петербурга москву, которая входит кремль воск

In [22]:
data2 = model.predict([text2])
df2 = pd.DataFrame(data2)
df2.insert(0, 'title', soup2.title.string)
df2

,title,neutral,negative,positive,skip,speech
0,Портреты русской цивилизации: Великий Новгород,0.461027,0.233716,0.125933,0.080367,0.028446


In [ ]:
# Снова ерунда получается. 
# Библиотека выдаёт, что этот текст менее нейтральный (0.461), и более негативный (0.233) чем новость 
# об отравившихся алкоголем туристах в Лаосе, где у нас было соответственно 0.644 и 0.228. 

Вывод: результаты работы библитеки dostoevsky вызывают у меня сомнения. 

#### Готово